In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
from keras import Model
from keras.layers import Dense , Flatten,Concatenate, Embedding,Input,Dropout
from keras.models import Sequential
from keras.regularizers import l1
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow import keras

seed = 2891  
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
from matplotlib import pyplot as plt
import warnings
import seaborn as sns 
warnings.filterwarnings('ignore')

C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_new = pd.read_csv('Train.csv')
band_names = [l.strip() for l in open('bandnames.txt', 'r').readlines()]
ss = pd.read_csv('SampleSubmission.csv')

In [3]:
def process_im(fid, folder='image_arrays_train'):
  fn = f'{folder}/{fid}.npy'
  arr = np.load(fn)
  bands_of_interest = ['S2_B1','S2_B2','S2_B3','S2_B4','S2_B5','S2_B6','S2_B7','S2_B8','S2_B8A','S2_B9','S2_B10',
                       'S2_B11','S2_B12','S2_QA10','S2_QA20' ,'S2_QA60','CLIM_aet','CLIM_def','CLIM_pdsi','CLIM_pet',
                       'CLIM_pr','CLIM_ro','CLIM_soil' ,'CLIM_srad' ,'CLIM_swe','CLIM_tmmn','CLIM_tmmx',
                       'CLIM_vap','CLIM_vpd','CLIM_vs']
  
  values = {}
  for month in range(12):
    bns = [str(month) + '_' + b for b in bands_of_interest] # Bands of interest for this month 
    idxs = np.where(np.isin(band_names, bns)) # Index of these bands
    vs = arr[idxs, 20, 20] # Sample the im at the center point
    for bn, v in zip(bns, vs[0]):
      values[bn] = v
  return values
def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['Yield'])
    y_predict = y_predict.join(t)   #Field_ID
    y_predic = y_predict[['Field_ID', 'Yield']]
    y_predic.to_csv(name_out + '.csv',index=False)
def input_fn_new(data_set, training = True):
    continuous_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
    
    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(data_set[k].size)], values = data_set[k].values, dense_shape = [data_set[k].size, 1]) for k in FEATURES_CAT}

    # Merges the two dictionaries into one.
    feature_cols = dict(list(continuous_cols.items()) + list(categorical_cols.items()))
    
    if training == True:
        # Converts the label column into a constant Tensor.
        label = tf.constant(data_set[LABEL].values)

        # Returns the feature columns and the label.
        return feature_cols, label
    
    return feature_cols
train_sampled = pd.DataFrame([process_im(fid) for fid in train_new['Field_ID'].values])

# Add in the field ID and yield
train_sampled['Field_ID'] = train_new['Field_ID'].values
train_sampled['Yield'] = train_new['Yield'].values
# train_sampled.head()
test_sampled = pd.DataFrame([process_im(fid, folder='image_arrays_test') for fid in ss['Field_ID'].values])
# test_sampled['Field_ID'] = ss['Field_ID']
# test_sampled['Yield'] = 0
# Example
# process_im('35AFSDD')

In [4]:
train = train_sampled.copy()
test = test_sampled.copy()
add_info = pd.read_csv('fields_w_additional_info.csv')

In [5]:
# add_info.isna().sum()
# A=add_info[add_info.columns[9]]
# Anan=A[~np.isnan(A)] # Remove the NaNs
# sns.distplot(Anan)
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer = imputer.fit(add_info)
# X= imputer.transform(add_info)
add_info[add_info.columns.tolist()[1:]] = add_info[add_info.columns.tolist()[1:]].apply(lambda x: x.fillna(x.mean()),axis=0)
prepo_add_info = MinMaxScaler()
add_info[add_info.columns.tolist()[1:]] = prepo_add_info.fit_transform(add_info[add_info.columns.tolist()[1:]])
# add_info.isna().sum()
add_info_train = add_info.loc[add_info['Field_ID'].isin(train.Field_ID.values)]
add_info_test =  add_info.loc[add_info['Field_ID'].isin(ss.Field_ID.values)]

In [6]:
# train.Field_ID
# add_info.where(add_info.Field_ID == train.Field_ID.values)
# add_info_test
add_info_cols = add_info.columns.tolist()[1:]

In [7]:

# # Import and split
# # train = pd.read_csv('train_df_cust.csv')
# train_numerical = train.select_dtypes(exclude=['object'])
# train_categoric = train.select_dtypes(include=['object'])
# train = train_numerical.merge(train_categoric, left_index = True, right_index = True) 

# # test = pd.read_csv('test_df_cust.csv')
# Yield_ID = ss.Field_ID
# test_numerical = test.select_dtypes(exclude=['object'])
# test_categoric = test.select_dtypes(include=['object'])
# test = test_numerical.merge(test_categoric, left_index = True, right_index = True) 
# # Removie the outliers

# clf = IsolationForest(max_samples = 100, random_state = 42)
# clf.fit(train_numerical)
# y_noano = clf.predict(train_numerical)
# y_noano = pd.DataFrame(y_noano, columns = ['Top'])
# y_noano[y_noano['Top'] == 1].index.values

# train_numerical = train_numerical.iloc[y_noano[y_noano['Top'] == 1].index.values]
# train_numerical.reset_index(drop = True, inplace = True)

# train_categoric = train_categoric.iloc[y_noano[y_noano['Top'] == 1].index.values]
# train_categoric.reset_index(drop = True, inplace = True)

# train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)

In [8]:
test_numerical = test.select_dtypes(exclude=['object'])
test_categoric = test.select_dtypes(include=['object'])
train_numerical = train.select_dtypes(exclude=['object'])
train_categoric = train.select_dtypes(include=['object'])
col_train = list(train.columns)
col_train_bis = list(train.columns)
col_train_bis.remove('Yield')

col_train_num = list(train_numerical.columns)
col_train_num_bis = list(train_numerical.columns)

col_train_cat = list(train_categoric.columns)

col_train_num_bis.remove('Yield')

mat_train = np.matrix(train_numerical)
mat_test  = np.matrix(test_numerical)
mat_new = np.matrix(train_numerical.drop('Yield',axis = 1))
mat_y = np.array(train.Yield)

In [9]:
# mat_test[2].shape
# 
# (mat_new ==  ) .sum()
# mat_train.shape

In [10]:
prepro_y = MinMaxScaler()
prepro_y.fit(mat_y.reshape(2977,1))

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_test)

train_num_scale = pd.DataFrame(prepro.transform(mat_train),columns = col_train_num)
test_num_scale  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_num_bis)

train[col_train_num] = pd.DataFrame(prepro.transform(mat_train),columns = col_train_num)
test[col_train_num_bis]  = test_num_scale

0       MH2O0YH
1       O9TURWL
2       35AFSDD
3       PM05EG9
4       V7PZBCG
         ...   
2972    FFX8W83
2973    JAOFQM9
2974    Z4D9MU8
2975    SX57GH1
2976    A69YGTK
Name: Field_ID, Length: 2977, dtype: object

In [17]:
# train['Yield']
Experiment_name="simple_model"
import os
os.makedirs("proc_data", exist_ok=True)
# os.makedirs("model_save/lgbm/{}".format(Experiment_name), exist_ok=True)
# os.makedirs("model_save/catboost/{}".format(Experiment_name), exist_ok=True)
# os.makedirs("model_save/xgboost/{}".format(Experiment_name), exist_ok=True)
try : 
    folds=pd.read_csv("./proc_data/folds_id.csv")
    train=train.merge(folds,on="Field_ID",how="left")
    train.fold.nunique()
except : 
    #  you run this cell  only for the first time 
    from sklearn.model_selection import KFold 
    kfold=KFold(n_splits=5,shuffle=True,random_state=2020) # change this random_state or all of you will have the same score  :D 
    train.reset_index(drop=True,inplace=True)
    folds=train[["Field_ID"]].copy()
    folds["fold"]=0
    for fold,(tr_indx,val_ind) in enumerate(kfold.split(folds)) : 
        folds.loc[val_ind,"fold"]=fold
    folds.to_csv("./proc_data/folds_id.csv",index=False)
    train=train.merge(folds,on="Field_ID",how="left")
    
    del folds

In [60]:
from sklearn.metrics import mean_squared_error
import  lightgbm as lgbm 
def metric(y,x):
    return np.sqrt(mean_squared_error(x,y))
def train_function(model,train,test,params,other_params,target_name,features,metric):
    folds_num=train.fold.nunique()
    validation=train[[id_name,"fold",target_name]].copy()
    validation["pred_"+target_name]=0
    sub=test[[id_name]].copy()
    sub[target_name]=0
    for fold in np.sort(train.fold.unique()):
        print("#"*50+" {} ".format(fold)+"#"*50)
        os.makedirs("model_save/lgbm/{}/{}".format(Experiment_name,str(int(fold))), exist_ok=True)
        X_train=train[train.fold!=fold]
        X_val=train[train.fold==fold]
        
        train_pred,validation_pred,test_pred=model(X_train,X_val,test,params,other_params)
        
        validation.loc[validation.fold==fold,"pred_"+target_name]=validation_pred
        sub[target_name]+=test_pred/folds_num
        train_score=metric(X_train[target_name],train_pred)
        val_score=metric(X_val[target_name],validation_pred)
        print("train score : {} validation score : {}".format(round(train_score,4),round(val_score,4)))
    final_validation_score=metric(validation[target_name],validation["pred_"+target_name])
    print("final validation score : {}".format(final_validation_score))
        
    return sub,validation,final_validation_score

def lgbm_model(X_train,X_val,X_test,params,other_params):
    dtrain = lgbm.Dataset(data=X_train[features], label=X_train[target_name], feature_name=features)
    dval = lgbm.Dataset(data=X_val[features], label=X_val[target_name], feature_name=features)

    model = lgbm.train(
        params=params,
        train_set=dtrain,
        num_boost_round=other_params["num_boost_round"],
        valid_sets=(dtrain, dval),
        early_stopping_rounds=other_params["early_stopping_rounds"],
        verbose_eval=other_params["verbose_eval"],
    )        
    best_iteration = model.best_iteration
    train_pred=model.predict(X_train[features], num_iteration=best_iteration)
    validation_pred=model.predict(X_val[features], num_iteration=best_iteration)
    test_pred=model.predict(test[features], num_iteration=best_iteration)
        
    return train_pred,validation_pred,test_pred

In [61]:
target_name="Yield"
id_name="Field_ID"
features_to_remove=[target_name,"fold"]
features=train.columns.tolist()
features=[ fea for fea in  features if fea not in features_to_remove  ]
test['Field_ID'] = ss.Field_ID
train = train.astype({"Field_ID":'category'})
test = test.astype({"Field_ID":'category'})


In [62]:
# train.Field_ID

In [71]:
# train.Yield
other_params={"num_boost_round":50000000,
              "early_stopping_rounds":50,
              "verbose_eval":1000,
}
lgbm_params = {
    "bagging_fraction": 0.8,
    "bagging_freq": 2,
    "boosting_type": "gbdt",
    "feature_fraction": 0.8,
    "learning_rate": 0.01,
    "max_depth": 8,
    "num_threads": 16,
    "objective": "regression",
    "metric": "rmse",
    "reg_lambda": 5,
    "seed": 2020,
}

In [72]:
sub,validation,score=train_function(model=lgbm_model,
                                    train=train,
                                    test=test,
                                    params=lgbm_params,
                                    other_params=other_params,
                                    target_name=target_name,
                                    features=features,
                                    metric=metric)

################################################## 0 ##################################################
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[200]	training's rmse: 0.0841254	valid_1's rmse: 0.111079
train score : 0.0841 validation score : 0.1111
################################################## 1 ##################################################
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[243]	training's rmse: 0.0808469	valid_1's rmse: 0.105704
train score : 0.0808 validation score : 0.1057
################################################## 2 ##################################################
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[252]	training's rmse: 0.0789019	valid_1's rmse: 0.111711
train score : 0.0789 validation score : 0.1117
################################################## 3 ################################

In [73]:
sub['Yield'] = prepro_y.inverse_transform(np.array(sub.Yield.values.reshape((sub.Yield.shape[0],1))))
# mat_y.shape

In [74]:
# i = 0

In [75]:
sub.to_csv(Experiment_name+str(i)+'.csv',index=False)
i = i+1

In [76]:
sub

,Field_ID,Yield
0,E9UZCEA,3.465100
1,1WGGS1Q,3.706014
2,EG2KXE2,3.641288
3,HC3GQXF,3.510425
4,7AK6GFK,3.526212
...,...,...
1050,3H89LWV,3.082509
1051,I6EYSGB,2.808866
1052,XOEIR44,2.203332
1053,YB307JG,2.539080
